<h1 style="text-align:center;font-size:30px;" > Image Segementation on Indian Driving Dataset </h1>

In [1]:
import cv2, os, random
import numpy as np
import shutil 
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as K

from tensorflow.keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [2]:
data = 'idd20k_lite1/'
img_test = data + 'leftImg8bit/val/'
seg_test = data + 'gtFine/val/'

test_img = os.listdir(img_test)
test_img.sort()
test_seg = os.listdir(seg_test)
test_seg.sort()

test_seg_label=[]
for i in range(len(test_seg)):
    if(i%2 !=0):
        test_seg_label.append(test_seg[i])

In [3]:
def final_fun_1(test_img):   # Takes input as parameter and returns output
    
    X_te=test_img

    # Resizing image height to 256 beacuse to send as image as inout to uNET model the height and width of the image should
    # multiples of 32
    height=256
    width=320
    n_classes=7
    def prepare_image_data(path,data):
        src=path+data
        img = cv2.imread(src)
        img=cv2.resize(img,(width,height))
        img = np.float32(img)  / 255          #normalization
        return img

    # https://github.com/advaitsave/Multiclass-Semantic-Segmentation-CamVid/blob/master/Multiclass%20Semantic%20Segmentation%20using%20U-Net.ipynb
    def prepare_label_data(path,data):
        label = np.zeros((height, width, n_classes))
        src=path+data
        img = cv2.imread(src)
        img=cv2.resize(img,(width,height))
        img1=img[:,:,0]
        for i in range(n_classes):
            label[:,:,i] = (img1==i).astype(int)
        return label
    
    X_test = []

    for i in range(len(X_te)):
        X_test.append(prepare_image_data(img_test,X_te[i]))
        
    X_test=np.array(X_test)
    
    #model
    #Performance metric
    def IoU(y_val, y_pred):
        class_iou = []
        n_classes = 7

        y_predi = np.argmax(y_pred, axis=3)
        y_truei = np.argmax(y_val, axis=3)

        for c in range(n_classes):
            TP = np.sum((y_truei == c) & (y_predi == c))
            FP = np.sum((y_truei != c) & (y_predi == c))
            FN = np.sum((y_truei == c) & (y_predi != c)) 
            IoU = TP / float(TP + FP + FN)
            if(float(TP + FP + FN) == 0):
                IoU=TP/0.001
            class_iou.append(IoU)
        MIoU=sum(class_iou)/n_classes
        return MIoU

    def miou( y_true, y_pred ) :
        score = tf.py_function( lambda y_true, y_pred : IoU( y_true, y_pred).astype('float32'),
                            [y_true, y_pred],
                            'float32')
        return score
    
    # the model being implemented is VGG16(Encoder)_Unet(decoder) which means
    # the encoder part of the Unet is being rplaced with the VGG16 encoder with weights trained on imagenet dataset
    
    dependencies = {
    'miou': miou
    }
    vgg_unet_model = load_model("vgg_unet_model2.h5", custom_objects = dependencies)
    
    y_pred = vgg_unet_model.predict(X_test)
    return y_pred

In [3]:
# Takes input and output as parameter and returns performance metric
def final_fun_2(test_img,test_seg_label):      
    
    X_te=test_img
    y_te=test_seg_label

    # Resizing image height to 256 beacuse to send as image as inout to uNET model the height and width of the image should
    # multiples of 32
    height=256
    width=320
    n_classes=7
    def prepare_image_data(path,data):
        src=path+data
        img = cv2.imread(src)
        img=cv2.resize(img,(width,height))
        img = np.float32(img)  / 255          #normalization
        return img

    # https://github.com/advaitsave/Multiclass-Semantic-Segmentation-CamVid/blob/master/Multiclass%20Semantic%20Segmentation%20using%20U-Net.ipynb
    def prepare_label_data(path,data):
        label = np.zeros((height, width, n_classes))
        src=path+data
        img = cv2.imread(src)
        img=cv2.resize(img,(width,height))
        img1=img[:,:,0]
        for i in range(n_classes):
            label[:,:,i] = (img1==i).astype(int)
        return label
    
    X_test, y_test = [], []

    for i in range(len(X_te)):
        X_test.append(prepare_image_data(img_test,X_te[i]))
        
    for i in range(len(y_te)):
        y_test.append(prepare_label_data(seg_test,y_te[i]))

    X_test=np.array(X_test)
    y_test=np.array(y_test)
    
    #model
    #Performance metric
    def IoU(y_val, y_pred):
        class_iou = []
        n_classes = 7

        y_predi = np.argmax(y_pred, axis=3)
        y_truei = np.argmax(y_val, axis=3)

        for c in range(n_classes):
            TP = np.sum((y_truei == c) & (y_predi == c))
            FP = np.sum((y_truei != c) & (y_predi == c))
            FN = np.sum((y_truei == c) & (y_predi != c)) 
            IoU = TP / float(TP + FP + FN)
            if(float(TP + FP + FN) == 0):
                IoU=TP/0.001
            class_iou.append(IoU)
        MIoU=sum(class_iou)/n_classes
        return MIoU

    def miou( y_true, y_pred ) :
        score = tf.py_function( lambda y_true, y_pred : IoU( y_true, y_pred).astype('float32'),
                            [y_true, y_pred],
                            'float32')
        return score
    
    # the model being implemented is VGG16(Encoder)_Unet(decoder) which means
    # the encoder part of the Unet is being rplaced with the VGG16 encoder with weights trained on imagenet dataset
    
    dependencies = {
    'miou': miou
    }
    vgg_unet_model = load_model("vgg_unet_model2.h5", custom_objects = dependencies)
        
    y_pred = vgg_unet_model.predict(X_test)
    metric_value=IoU(y_test, y_pred)
    return metric_value

In [4]:
print('MIoU for VGG16_UNet model with Image augmentation is :',final_fun_2(test_img,test_seg_label))

MIoU for VGG16_UNet model with Image augmentation is : 0.560029946833321
